# Spleeter Separation

In [ ]:
import os
import glob
import time
import keras
from tqdm import tqdm
import numpy as np
from spleeter.audio import Codec, STFTBackend
from spleeter.audio.adapter import AudioAdapter
from spleeter.separator import Separator
from IPython.display import display, Audio

### Separation Function

In [ ]:
def separate(in_file, model):
    """
    Run Spleeter source separation model on WAV files to isolate stems.

    :param in_file: (str) path of mixture WAV file to separate
    :param model: (spleeter.separator.Separator) spleeter model object
    """  
    # only process wav files
    if in_file.endswith(".wav"):
        # read the soundfile
        waveform, _ = ADAPTER.load(in_file, sample_rate=SAMPLE_RATE)
    
        out = model.separate(waveform)
        out_stems = []
        for s in STEMS:
            est_stem = out[s]
            out_stems.append(est_stem)

    else:
        out_stems = None
        print("Invalid input file type. Please use WAV files only")

    return out_stems

### Separate Stereo Mixtures

In [ ]:
# set input and output directories
INPUT_DIR = "../data/musdb18hq/test/"
OUTPUT_DIR = "../data/output/spleeter_test/stereo/test/"

In [ ]:
STEMS = ["drums", "bass", "other", "vocals"]
SAMPLE_RATE = 44100

In [ ]:
# load the model
MODEL = Separator('spleeter:4stems', MWF=False, stft_backend=STFTBackend.AUTO, multiprocess=False)

In [ ]:
# set up audio adapter
adapter_type = "spleeter.audio.ffmpeg.FFMPEGProcessAudioAdapter"
ADAPTER = AudioAdapter.get(adapter_type)
BITRATE = "128k"
CODEC = Codec.WAV

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
file_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, f))]
print(f"There are {len(file_list)} files in the input directory.")

In [ ]:
# create the output directory if it does not already exist
print("Creating output directory, if it does not already exist...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# iterate through each file
print("Beginning to process files...")
for file in tqdm(file_list):
    # print(f"\n{file}")
    out_dir = os.path.join(OUTPUT_DIR, file)
    os.makedirs(out_dir, exist_ok=True)
    out = separate(os.path.join(INPUT_DIR, file, 'mixture.wav'), MODEL)
    for i in range(len(STEMS)):
        out_path = os.path.join(out_dir, STEMS[i] + '.wav')
        ADAPTER.save(out_path, out[i], SAMPLE_RATE, CODEC, BITRATE)
print("Processing complete!")

### Separate Standard Binaural Mixtures

In [ ]:
# set input and output directories
INPUT_DIR = "../data/binaural_musdb18/standard/test/"
OUTPUT_DIR = "../data/output/spleeter_test/standard/test/"

In [ ]:
STEMS = ["drums", "bass", "other", "vocals"]
SAMPLE_RATE = 44100

In [ ]:
# load the model
MODEL = Separator('spleeter:4stems', MWF=False, stft_backend=STFTBackend.AUTO, multiprocess=False)

In [ ]:
# set up audio adapter
adapter_type = "spleeter.audio.ffmpeg.FFMPEGProcessAudioAdapter"
ADAPTER = AudioAdapter.get(adapter_type)
BITRATE = "128k"
CODEC = Codec.WAV

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
file_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, f))]
print(f"There are {len(file_list)} files in the input directory.")

In [ ]:
# create the output directory if it does not already exist
print("Creating output directory, if it does not already exist...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# iterate through each file
print("Beginning to process files...")
for file in tqdm(file_list):
    out_dir = os.path.join(OUTPUT_DIR, file)
    os.makedirs(out_dir, exist_ok=True)
    out = separate(os.path.join(INPUT_DIR, file, 'mixture.wav'), MODEL)
    for i in range(len(STEMS)):
        out_path = os.path.join(out_dir, STEMS[i] + '.wav')
        ADAPTER.save(out_path, out[i], SAMPLE_RATE, CODEC, BITRATE)
print("Processing complete!")

### Separate Random Binaural Mixtures

In [ ]:
# set input and output directories
INPUT_DIR = "../data/binaural_musdb18/random/test/"
OUTPUT_DIR = "../data/output/spleeter_test/random/test/"

In [ ]:
STEMS = ["drums", "bass", "other", "vocals"]
SAMPLE_RATE = 44100

In [ ]:
# load the model
MODEL = Separator('spleeter:4stems', MWF=False, stft_backend=STFTBackend.AUTO, multiprocess=False)

In [ ]:
# get all of the files in the input directory
print("Loading list of files...")
file_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, f))]
print(f"There are {len(file_list)} files in the input directory.")

In [ ]:
# create the output directory if it does not already exist
print("Creating output directory, if it does not already exist...")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# iterate through each file
print("Beginning to process files...")
for file in tqdm(file_list):
    # print(f"\n{file}")
    out_dir = os.path.join(OUTPUT_DIR, file)
    os.makedirs(out_dir, exist_ok=True)
    out = separate(os.path.join(INPUT_DIR, file, 'mixture.wav'), MODEL)
    for i in range(len(STEMS)):
        out_path = os.path.join(out_dir, STEMS[i] + '.wav')
        ADAPTER.save(out_path, out[i], SAMPLE_RATE, CODEC, BITRATE)
print("Processing complete!")

## Using SPAUQ Function

[Source](https://github.com/kwatcharasupat/spauq/blob/b9bca9cfb426e93d2f59e664c1689efc537bf63e/expt/music-source-seperation/inference.py#L11)

In [ ]:
import glob
import os
from tqdm import tqdm

In [ ]:
def inference_spleeter(variant, audio_path, output_path):
    assert variant in ["4stems"]

    from spleeter.audio import Codec, STFTBackend
    from spleeter.audio.adapter import AudioAdapter
    from spleeter.separator import Separator

    mixtures = glob.glob(
        os.path.join(audio_path, "**", "mixture.wav"), recursive=True
    )
    os.makedirs(output_path, exist_ok=True)

    files = mixtures
    adapter = "spleeter.audio.ffmpeg.FFMPEGProcessAudioAdapter"
    bitrate = "128k"
    codec = Codec.WAV
    duration = 600.0
    offset = 0
    output_path = output_path
    stft_backend = STFTBackend.AUTO
    filename_format = "{foldername}/{instrument}.{codec}"
    params_filename = f"spleeter:{variant}"
    mwf = False
    verbose = False

    audio_adapter = AudioAdapter.get(adapter)
    separator = Separator(params_filename, MWF=mwf,
                          stft_backend=stft_backend,
                          multiprocess=False)

    for filename in tqdm(files):
        separator.separate_to_file(
            str(filename),
            str(output_path),
            audio_adapter=audio_adapter,
            offset=offset,
            duration=duration,
            codec=codec,
            bitrate=bitrate, 
            filename_format=filename_format,
            synchronous=True,
        )

**Stereo Mixtures**

In [ ]:
# set input and output directories
AUDIO_DIR = "../data/musdb18hq/test/"
OUTPUT_DIR = "../data/output/spleeter_test/stereo/test/"

In [ ]:
inference_spleeter('4stems', AUDIO_DIR, OUTPUT_DIR)

**Standard Binaural Mixtures**

In [ ]:
# set input and output directories
AUDIO_DIR = "../data/binaural_musdb18/standard/test/"
OUTPUT_DIR = "../data/output/spleeter_test/standard/test/"

In [ ]:
inference_spleeter('4stems', AUDIO_DIR, OUTPUT_DIR)

**Random Binaural Mixtures**

In [ ]:
# set input and output directories
AUDIO_DIR = "../data/binaural_musdb18/random/test/"
OUTPUT_DIR = "../data/output/spleeter_test/random/test/"

In [ ]:
inference_spleeter('4stems', AUDIO_DIR, OUTPUT_DIR)